<p align="center">
  <img width="550" height="281" src="http://encuentratubeca.mx/wp-content/uploads/2019/04/logoudgvectores_1_origsdaasss.png">
</p>

# <center>Proyecto Final</center>             
------
## Maestría en Ciencias de Datos
#### Alberto Martinez
#### Mauricio Tenorio

#### Marzo  2023

In [2]:
# Importamos librerias necesarias para el analisis de los datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import altair as alt

#### Informacion del Dataset
Dataset utilizando 

Variables:

In [3]:
#Descargamos dataset directamente desde el url
path = "https://raw.githubusercontent.com/AlbertoMCD/ChatGPT_tweets/main/data/GPT.csv"
df = pd.read_csv(path)

In [4]:
df.head()

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,RT @gdb: ChatGPT as a tool for Congress: https...,False,0,NaN,4/27/2023 17:35:29,False,NaN,1651641228520194048,NaN,"<a href=""http://twitter.com/download/iphone"" r...",nos_ult,58,True,False,NaN,NaN
1,RT @gdb: ChatGPT as a tool for Congress: https...,False,0,NaN,4/27/2023 17:35:28,False,NaN,1651641223428317185,NaN,"<a href=""http://twitter.com/download/android"" ...",verobouvier,58,True,False,NaN,NaN
2,RT @moemicky18: So chatgpt failed an accountin...,False,0,NaN,4/27/2023 17:35:27,False,NaN,1651641221893308416,NaN,"<a href=""http://twitter.com/download/iphone"" r...",uzayrmug,2,True,False,NaN,NaN
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,False,0,NaN,4/27/2023 17:35:27,False,NaN,1651641218059608094,NaN,"<a href=""http://twitter.com/download/iphone"" r...",SachaMery,359,True,False,NaN,NaN
4,"does chatgpt mean I, as a wet lab biologist wh...",False,0,NaN,4/27/2023 17:35:26,True,NaN,1651641214557364234,NaN,"<a href=""http://twitter.com/download/android"" ...",ruth_hook_,0,False,False,NaN,NaN


In [5]:
df.shape

(3000, 16)

### Data Cleaning

In [6]:
df.isnull().sum()

text                0
favorited           0
favoriteCount       0
replyToSN        2629
created             0
truncated           0
replyToSID       2640
id                  0
replyToUID       2629
statusSource        0
screenName          0
retweetCount        0
isRetweet           0
retweeted           0
longitude        2998
latitude         2998
dtype: int64